In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
pip install pymupdf pdfplumber pdf2image pytesseract pandas opencv-python numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 78.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 73.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 63.6 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pdfplumber
import pytesseract
import cv2
import numpy as np
import os
import pandas as pd
from PIL import Image

# 📍 Chemin du PDF (Téléchargez un fichier dans Kaggle avant)
pdf_path = "/kaggle/input/volume1/Synthse volume I (3).pdf"

# 📍 Répertoire pour stocker les images extraites
image_output_dir = "/kaggle/working/extracted_images"
os.makedirs(image_output_dir, exist_ok=True)

# 📌 1️⃣ Extraction du texte du PDF (Sans Poppler)
def extract_text_from_pdf(pdf_path):
    text_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text:
                text_data.append({"page": i + 1, "text": text})
    
    return text_data

# 📌 2️⃣ Extraction des images et OCR pour texte scanné
def extract_images_and_text(pdf_path, image_output_dir):
    images_text_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            # 📍 Extraire les images de la page
            for img_index, img in enumerate(page.images):
                image_path = f"{image_output_dir}/page_{i+1}_img_{img_index+1}.jpg"
                
                # Convertir en format d'image PIL
                image = page.to_image().annotated
                image.save(image_path, "JPEG")

                # Charger l'image avec OpenCV
                img_cv = cv2.imread(image_path)
                gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
                
                # Améliorer l'image pour OCR (binarisation)
                _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                # Reconnaissance de texte (OCR)
                text = pytesseract.image_to_string(thresh, lang="eng")

                images_text_data.append({"page": i + 1, "image_path": image_path, "text": text.strip()})
    
    return images_text_data

# 📌 3️⃣ Segmentation des contenus : Titres et Paragraphes
def segment_text(text_data):
    structured_data = []

    for page_data in text_data:
        page_number = page_data["page"]
        text = page_data["text"]

        # Séparer les titres et paragraphes (Ex: Titres en MAJUSCULES)
        lines = text.split("\n")
        sections = []
        current_section = {"title": None, "content": ""}

        for line in lines:
            if line.strip().isupper():  # Détection des titres
                if current_section["title"]:
                    sections.append(current_section)
                current_section = {"title": line.strip(), "content": ""}
            else:
                current_section["content"] += line + " "
        
        if current_section["title"]:
            sections.append(current_section)

        structured_data.append({"page": page_number, "sections": sections})
    
    return structured_data

# 📌 4️⃣ Extraction des tableaux du PDF
def extract_tables_from_pdf(pdf_path):
    tables_data = []

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            tables = page.extract_tables()
            if tables:
                for table in tables:
                    df = pd.DataFrame(table)  # Convertir en DataFrame
                    tables_data.append({"page": i + 1, "table": df})
    
    return tables_data

# 📌 Exécution des étapes
print("\n🔹 Extraction du texte...")
text_data = extract_text_from_pdf(pdf_path)

print("\n🔹 Extraction des images et OCR...")
images_text_data = extract_images_and_text(pdf_path, image_output_dir)

print("\n🔹 Segmentation des contenus...")
structured_text = segment_text(text_data)

print("\n🔹 Extraction des tableaux...")
tables_data = extract_tables_from_pdf(pdf_path)

# 📌 Affichage des résultats
print("\n🔹 Texte structuré extrait :")
for page in structured_text:
    print(f"\n📄 Page {page['page']}:")
    for section in page["sections"]:
        print(f"\n📝 {section['title']}\n{section['content']}")

print("\n🔹 Texte extrait des images :")
for img_data in images_text_data:
    print(f"\n📷 Image page {img_data['page']} ({img_data['image_path']}):\n{img_data['text']}")

print("\n🔹 Tableaux extraits :")
for table_data in tables_data:
    print(f"\n📊 Table sur la page {table_data['page']}:")
    print(table_data["table"])



🔹 Extraction du texte...

🔹 Extraction des images et OCR...

🔹 Segmentation des contenus...

🔹 Extraction des tableaux...

🔹 Texte structuré extrait :

📄 Page 1:

📝 UNIVERSITE DE TUNIS


📝 FACULTE DES SCIENCES HUMAINES ET SOCIALES DE TUNIS


📝 S


📝 VFES ET SA REGION DANS


📝 L9ANTIQUITE


📝 V I


📝 OLUME


📝 T


📝 HESE DE DOCTORAT


📝 H A


📝 EN ISTOIRE NCIENNE
Préparée par Sous la direction du Professeur Mohamed GRIRA Ahmed M9CHAREK 

📝 T 2008


📝 UNIS


📄 Page 2:

📝 UNIVERSITE DE TUNIS


📝 FACULTE DES SCIENCES HUMAINES ET SOCIALES DE TUNIS


📝 S


📝 VFES (S )


📝 BIBA ET SA REGION


📝 DANS L9ANTIQUITE


📝 V I


📝 OLUME


📝 T


📝 HESE DE DOCTORAT


📝 H A


📝 EN ISTOIRE NCIENNE
Préparée par Sous la direction du Professeur Mohamed GRIRA Ahmed M9CHAREK 

📝 T 2008


📝 UNIS


📄 Page 3:

📄 Page 4:

📝 REMERCIEMENT
Il m9est agréable d9adresser mes sincères remerciements en premier lieu à mon directeur de recherches Mr. le Professeur Ahmed M9Charek pour ses encouragements et sa générosité sc

In [25]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [27]:
import spacy
import os

# 📌 Vérifier si le modèle français est installé, sinon l'installer automatiquement
try:
    nlp = spacy.load("fr_core_news_sm")
except OSError:
    print("🔹 Modèle 'fr_core_news_sm' non trouvé. Installation en cours...")
    os.system("pip install spacy")
    os.system("python -m spacy download fr_core_news_sm")
    nlp = spacy.load("fr_core_news_sm")  # Charger après installation

print("✅ Modèle spaCy 'fr_core_news_sm' chargé avec succès !")


🔹 Modèle 'fr_core_news_sm' non trouvé. Installation en cours...


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


✅ Modèle spaCy 'fr_core_news_sm' chargé avec succès !


In [28]:
import os
import re
import spacy
import nltk
import pandas as pd
import cv2
import numpy as np
import pytesseract
from symspellpy import SymSpell
from nltk.tokenize import sent_tokenize
from tabula import read_pdf

# 📌 Initialisation des outils
nlp = spacy.load("fr_core_news_sm")  # NLP pour détecter les titres et paragraphes en français
nltk.download("punkt")  # Télécharge le tokenizer pour la segmentation de texte

# 📌 📍 Générer un dictionnaire français personnalisé pour SymSpell
def create_french_dictionary():
    """Crée un dictionnaire de correction orthographique basé sur les mots français les plus fréquents."""
    dictionary_path = "/kaggle/working/dictionnaire_francais.txt"

    # 📌 Liste des mots les plus courants en français (vous pouvez l'enrichir)
    common_words = [
        "le", "la", "les", "un", "une", "des", "du", "au", "aux", "ce", "cette",
        "ces", "mon", "ton", "son", "notre", "votre", "leur", "je", "tu", "il",
        "elle", "nous", "vous", "ils", "elles", "être", "avoir", "faire", "dire",
        "pouvoir", "aller", "voir", "savoir", "vouloir", "venir", "devoir", "trouver",
        "donner", "prendre", "parler", "aimer", "passer", "mettre", "demander",
        "penser", "laisser", "arriver", "entendre", "comprendre", "rendre"
    ]

    # 📌 Création du fichier dictionnaire avec une fréquence élevée (10 000)
    with open(dictionary_path, "w", encoding="utf-8") as file:
        for word in common_words:
            file.write(f"{word} 10000\n")  # Fréquence élevée pour donner priorité à ces mots

    return dictionary_path

# 📌 📍 Création du fichier dictionnaire français
dictionary_path = create_french_dictionary()

# 📌 Vérification que le fichier dictionnaire existe
if not os.path.exists(dictionary_path):
    print(f"❌ ERREUR : Le fichier dictionnaire n'a pas été créé.")
else:
    print(f"✅ Dictionnaire créé avec succès à {dictionary_path}.")

# 📌 Configuration de SymSpell avec le dictionnaire français
sym_spell = SymSpell(max_dictionary_edit_distance=2)
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# 📌 Fonction de correction OCR
def correct_ocr_errors(text):
    """Corrige les erreurs OCR avec SymSpell en français."""
    words = text.split()
    corrected_words = [
        sym_spell.lookup(word, verbosity=0)[0].term if sym_spell.lookup(word, verbosity=0) else word for word in words
    ]
    return " ".join(corrected_words)

# 📌 Exemple d'utilisation
sample_text = "Jaim baucoup ce que vous avés fais avc votre projet."
corrected_text = correct_ocr_errors(sample_text)
print(f"🔹 Texte avant correction OCR: {sample_text}")
print(f"✅ Texte après correction OCR: {corrected_text}")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
✅ Dictionnaire créé avec succès à /kaggle/working/dictionnaire_francais.txt.
🔹 Texte avant correction OCR: Jaim baucoup ce que vous avés fais avc votre projet.
✅ Texte après correction OCR: Jaim baucoup ce une vous avés faire au votre projet.


In [ ]:
pip install symspellpy tabula-py


In [33]:
import spacy
import torch

# 📌 Vérifier si un GPU est disponible et activer spaCy sur GPU
if torch.cuda.is_available():
    print("✅ GPU détecté. Activation du GPU pour spaCy...")
    spacy.require_gpu()  # Active le GPU si disponible
else:
    print("⚠️ Aucun GPU détecté. spaCy s'exécutera sur CPU.")

# 📌 Chargement du modèle spaCy français
try:
    nlp = spacy.load("fr_core_news_sm")
except OSError:
    print("🔹 Modèle 'fr_core_news_sm' non trouvé. Installation en cours...")
    os.system("python -m spacy download fr_core_news_sm")
    nlp = spacy.load("fr_core_news_sm")

print("✅ Modèle spaCy 'fr_core_news_sm' chargé avec succès !")

✅ GPU détecté. Activation du GPU pour spaCy...
✅ Modèle spaCy 'fr_core_news_sm' chargé avec succès !


In [37]:
pip install spacy nltk pandas opencv-python pytesseract symspellpy tabula-py

Note: you may need to restart the kernel to use updated packages.


In [38]:
python -m spacy download fr_core_news_sm

SyntaxError: invalid syntax (<ipython-input-38-0c3514199acd>, line 1)

In [44]:
import os
import re
import spacy
import nltk
import pandas as pd
import cv2
import numpy as np
import pytesseract
from symspellpy import SymSpell
from nltk.tokenize import sent_tokenize
from tabula import read_pdf

# 📌 📍 Définition des chemins
pdf_path = "/kaggle/input/volume1/Synthse volume I (3).pdf"  # Remplacez avec le bon chemin
image_folder = "/kaggle/working/extracted_images"  # Dossier des images extraites

# 📌 Vérification que le fichier PDF existe
if not os.path.exists(pdf_path):
    print(f"❌ ERREUR : Le fichier PDF n'existe pas à {pdf_path}. Vérifiez son emplacement sur Kaggle.")
else:
    print(f"✅ PDF trouvé : {pdf_path}")

# 📌 📍 Vérification du GPU et activation pour spaCy
import torch
if torch.cuda.is_available():
    print("✅ GPU détecté. Activation du GPU pour spaCy...")
    spacy.require_gpu()
else:
    print("⚠️ Aucun GPU détecté. spaCy s'exécutera sur CPU.")

# 📌 📍 Chargement du modèle spaCy en français
try:
    nlp = spacy.load("fr_core_news_sm")
except OSError:
    print("🔹 Modèle 'fr_core_news_sm' non trouvé. Installation en cours...")
    os.system("python -m spacy download fr_core_news_sm")
    nlp = spacy.load("fr_core_news_sm")

# 📌 Télécharger les données NLTK
nltk.download("punkt")

# 📌 📍 Création d’un dictionnaire français pour SymSpell
def create_french_dictionary():
    dictionary_path = "/kaggle/working/dictionnaire_francais.txt"
    common_words = ["le", "la", "les", "un", "une", "des", "du", "au", "aux", "ce", "cette",
                    "ces", "mon", "ton", "son", "notre", "votre", "leur", "je", "tu", "il",
                    "elle", "nous", "vous", "ils", "elles", "être", "avoir", "faire", "dire",
                    "pouvoir", "aller", "voir", "savoir", "vouloir", "venir", "devoir", "trouver",
                    "donner", "prendre", "parler", "aimer", "passer", "mettre", "demander",
                    "penser", "laisser", "arriver", "entendre", "comprendre", "rendre"]

    with open(dictionary_path, "w", encoding="utf-8") as file:
        for word in common_words:
            file.write(f"{word} 10000\n")

    return dictionary_path

dictionary_path = create_french_dictionary()

# 📌 Charger le dictionnaire en UTF-8
sym_spell = SymSpell(max_dictionary_edit_distance=2)
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1, encoding="utf-8")

# 📌 1️⃣ Nettoyage et correction OCR
def clean_text(text):
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^\w\s]", "", text)
    return text.strip()

def correct_ocr_errors(text):
    words = text.split()
    corrected_words = [sym_spell.lookup(word, verbosity=0)[0].term if sym_spell.lookup(word, verbosity=0) else word for word in words]
    return " ".join(corrected_words)

# 📌 2️⃣ Segmentation des Titres et Paragraphes
def segment_text(text_data):
    structured_data = []
    for page_data in text_data:
        page_number = page_data["page"]
        text = clean_text(page_data["text"])
        text = correct_ocr_errors(text)

        doc = nlp(text)
        sections = []
        current_section = {"title": None, "content": ""}

        for sent in doc.sents:
            sentence = sent.text.strip()
            if sentence.isupper():
                if current_section["title"]:
                    sections.append(current_section)
                current_section = {"title": sentence, "content": ""}
            else:
                current_section["content"] += sentence + " "
        
        if current_section["title"]:
            sections.append(current_section)

        structured_data.append({"page": page_number, "sections": sections})
    
    return structured_data

# 📌 3️⃣ Extraction et Nettoyage des Tableaux avec Pandas
def extract_and_clean_tables(pdf_path):
    tables = []
    try:
        extracted_tables = read_pdf(pdf_path, pages="all", multiple_tables=True)
        
        # 📌 Nettoyage des valeurs NaN et infinies
        cleaned_tables = []
        for df in extracted_tables:
            df.replace([np.inf, -np.inf], np.nan, inplace=True)
            
            # 📌 Convertir explicitement toutes les colonnes en `str` avant `fillna("")`
            df = df.astype(str)
            df.fillna("", inplace=True)
            
            cleaned_tables.append(df)

        tables.extend(cleaned_tables)

    except Exception as e:
        print(f"⚠️ Erreur Tabula : {e}")
    
    return tables

# 📌 4️⃣ Détection du Texte dans les Images avec Tesseract
def detect_text_in_images(image_folder):
    extracted_text = []
    
    if not os.path.exists(image_folder):
        print(f"❌ ERREUR : Le dossier {image_folder} n'existe pas.")
        return []

    image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith(('.png', '.jpg', '.jpeg'))]

    if not image_paths:
        print("⚠️ Aucune image trouvée dans le dossier.")
        return []

    for image_path in image_paths:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"⚠️ Impossible de charger l'image : {image_path}. Fichier ignoré.")
            continue  

        try:
            text = pytesseract.image_to_string(img, lang="fra")
            extracted_text.append({"image": image_path, "text": text.strip()})
        except Exception as e:
            print(f"⚠️ Erreur OCR sur l'image {image_path}: {e}")
            continue

    return extracted_text

# 📌 Exécution des Étapes
print("\n🔹 Nettoyage et segmentation du texte...")
structured_text = segment_text(text_data)  # ⚠️ `text_data` doit être extrait avant

print("\n🔹 Extraction et nettoyage des tableaux...")
tables_data = extract_and_clean_tables(pdf_path)

print("\n🔹 Traitement des images et extraction de texte...")
image_text_data = detect_text_in_images(image_folder)

# 📌 Affichage des Résultats
print("\n🔹 Tableaux extraits et nettoyés :")
for i, df in enumerate(tables_data):
    print(f"\n📊 Table {i+1}:")
    print(df)

print("\n🔹 Texte extrait des images :")
for img_data in image_text_data:
    print(f"\n📷 Image ({img_data['image']}):\n{img_data['text']}")


✅ PDF trouvé : /kaggle/input/volume1/Synthse volume I (3).pdf
✅ GPU détecté. Activation du GPU pour spaCy...
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!

🔹 Nettoyage et segmentation du texte...

🔹 Extraction et nettoyage des tableaux...

🔹 Traitement des images et extraction de texte...

🔹 Tableaux extraits et nettoyés :

📊 Table 1:
Empty DataFrame
Columns: [Fig. 2 : Les grands axes montagneux de la région de Sbiba]
Index: []

📊 Table 2:
Empty DataFrame
Columns: [Fig. 18 : la carte administrative de l9époque coloniale]
Index: []

📊 Table 3:
               Cités Distance
0     Sufes-Sufetula    35 km
1        Sufes-Thala    40 km
2       Sufes-Mididi    28 km
3      Sufes-Thugga-    35 km
4       Terebenthina      nan
5                nan      221
6  Sufes-Marazanae ?   35 km?
7   Sufes-Masclianae    41 km

📊 Table 4:
             2Espace Superficie en km Unnamed: 0 Nombre des sites  \
0              Vallée de Sgerm

In [39]:
import os

# 📌 Installation des dépendances avec os.system()
os.system("pip install spacy nltk pandas opencv-python pytesseract symspellpy tabula-py")
os.system("python -m spacy download fr_core_news_sm")
os.system("apt-get update")
os.system("apt-get install -y tesseract-ocr tesseract-ocr-fra")


0

In [45]:
pip install spacy pandas scikit-learn transformers torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [48]:
import os
import spacy
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision import models
from torchvision.models import ResNet18_Weights  # ✅ Correction pour ResNet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords

# 📌 📍 Téléchargement des stopwords en français
nltk.download('stopwords')
french_stopwords = stopwords.words("french")

# 📌 📍 Activation du GPU pour spaCy (évite les warnings)
if torch.cuda.is_available():
    print("✅ GPU détecté. Activation du GPU pour spaCy...")
    spacy.require_gpu()
else:
    print("⚠️ Aucun GPU détecté. spaCy s'exécutera sur CPU.")

# 📌 📍 Chargement du modèle spaCy pour le français (NER)
nlp = spacy.load("fr_core_news_sm")

# ==========================================
# 1️⃣ CATEGORISATION DU TEXTE (Correction `n_init`)
# ==========================================
def categorize_text_kmeans(texts, n_clusters=5):
    """Catégorise le texte en clusters avec TF-IDF et K-Means."""
    vectorizer = TfidfVectorizer(stop_words=french_stopwords)  # ✅ Utilisation correcte des stopwords en français
    X = vectorizer.fit_transform(texts)
    
    model = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)  # ✅ Correction du warning `n_init`
    labels = model.fit_predict(X)
    
    return labels

def extract_entities(text):
    """Extrait les entités nommées (NER) à partir du texte."""
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

def label_text_sections(text_data):
    """Attribue des labels aux paragraphes et les regroupe par thème."""
    texts = [section["content"] for page in text_data for section in page["sections"] if section["content"]]
    
    if not texts:
        print("⚠️ Aucune donnée de texte à analyser.")
        return pd.DataFrame()

    # 📌 Clustering des textes avec TF-IDF + KMeans
    clusters = categorize_text_kmeans(texts, n_clusters=5)
    
    # 📌 Extraction des entités avec spaCy (NER)
    labeled_data = []
    index = 0  # Indice pour suivre les textes
    for page in text_data:
        for section in page["sections"]:
            if section["content"]:
                section_text = section["content"]
                entities = extract_entities(section_text)
                labeled_data.append({
                    "page": page["page"],
                    "title": section["title"],
                    "content": section_text,
                    "cluster": int(clusters[index]),
                    "entities": entities
                })
                index += 1  # Incrémentation pour suivre le bon cluster
    
    return pd.DataFrame(labeled_data)

# ==========================================
# 2️⃣ STRUCTURATION DES TABLEAUX
# ==========================================
def structure_tables(tables_data):
    """Convertit les tableaux en CSV et JSON et vérifie la structure."""
    structured_tables = []
    
    for i, df in enumerate(tables_data):
        if df.empty:
            print(f"⚠️ Table {i+1} est vide. Ignorée.")
            continue
        
        # 📌 Enregistrement en CSV et JSON
        csv_path = f"/kaggle/working/table_{i+1}.csv"
        json_path = f"/kaggle/working/table_{i+1}.json"
        df.to_csv(csv_path, index=False)
        df.to_json(json_path, orient="records")
        
        # 📌 Vérification des colonnes
        columns = df.columns.tolist()
        structured_tables.append({"table": i+1, "columns": columns, "csv_path": csv_path, "json_path": json_path})
    
    return structured_tables

# ==========================================
# 3️⃣ CLASSIFICATION DES IMAGES AVEC CNN (Correction `pretrained`)
# ==========================================
def classify_images(image_folder):
    """Classifie les images en catégories avec ResNet."""
    model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)  # ✅ Correction du warning `pretrained`
    model.eval()
    model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    image_labels = []
    
    if not os.path.exists(image_folder):
        print(f"❌ ERREUR : Le dossier {image_folder} n'existe pas.")
        return []
    
    image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith(('.png', '.jpg', '.jpeg'))]

    if not image_paths:
        print("⚠️ Aucune image trouvée dans le dossier.")
        return []

    for image_path in image_paths:
        from PIL import Image
        image = Image.open(image_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
        
        with torch.no_grad():
            output = model(image)
            predicted_class = torch.argmax(output).item()
        
        image_labels.append({"image": image_path, "label": predicted_class})
    
    return image_labels

# ==========================================
# 4️⃣ ASSOCIATION TEXTE - IMAGE
# ==========================================
def associate_images_with_text(image_labels, text_data):
    """Associe chaque image avec un texte basé sur le contexte (même page)."""
    associations = []
    for img in image_labels:
        image_path = img["image"]
        predicted_label = img["label"]

        # 📌 Trouver le texte correspondant basé sur le clustering
        text_info = [text for text in text_data if text["cluster"] == predicted_label]
        
        associations.append({
            "image": image_path,
            "label": predicted_label,
            "associated_text": text_info[:3]  # Renvoie 3 textes similaires
        })
    
    return associations

# ==========================================
# 📌 📍 EXECUTION DE L'ETAPE 3
# ==========================================
print("\n🔹 Catégorisation du texte...")
labeled_text_df = label_text_sections(structured_text)

print("\n🔹 Structuration des tableaux...")
structured_tables = structure_tables(tables_data)

print("\n🔹 Classification des images...")
image_labels = classify_images("/kaggle/working/extracted_images")

print("\n🔹 Association texte - image...")
image_text_associations = associate_images_with_text(image_labels, labeled_text_df.to_dict(orient="records"))


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
✅ GPU détecté. Activation du GPU pour spaCy...

🔹 Catégorisation du texte...

🔹 Structuration des tableaux...
⚠️ Table 1 est vide. Ignorée.
⚠️ Table 2 est vide. Ignorée.
⚠️ Table 19 est vide. Ignorée.
⚠️ Table 32 est vide. Ignorée.
⚠️ Table 41 est vide. Ignorée.
⚠️ Table 77 est vide. Ignorée.
⚠️ Table 81 est vide. Ignorée.

🔹 Classification des images...

🔹 Association texte - image...


In [52]:
import pandas as pd
import json
import os
import shutil

# 📌 📍 Définition des chemins pour les fichiers exportés
output_dir = "/kaggle/working/exported_data"
zip_path = "/kaggle/working/data_export.zip"
os.makedirs(output_dir, exist_ok=True)  # Crée le dossier s'il n'existe pas

# ==========================================
# ✅ EXPORTATION DES DONNÉES EN CSV & JSON
# ==========================================
def export_data(labeled_text_df, tables_data, image_text_associations):
    """Exporte les données structurées en CSV et JSON et les place dans un dossier."""

    # 📌 Exportation des textes labellisés
    text_csv_path = os.path.join(output_dir, "labeled_text.csv")
    text_json_path = os.path.join(output_dir, "labeled_text.json")

    labeled_text_df.to_csv(text_csv_path, index=False)
    labeled_text_df.to_json(text_json_path, orient="records", force_ascii=False)

    print(f"✅ Textes exportés : 📂 {text_csv_path}")

    # 📌 Exportation des tableaux structurés
    tables_json_path = os.path.join(output_dir, "tables.json")
    tables_json = [df.to_dict(orient="records") for df in tables_data]

    with open(tables_json_path, "w", encoding="utf-8") as f:
        json.dump(tables_json, f, ensure_ascii=False, indent=4)

    print(f"✅ Tables exportées : 📂 {tables_json_path}")

    # 📌 Exportation des associations Texte-Image
    images_json_path = os.path.join(output_dir, "image_text_associations.json")
    with open(images_json_path, "w", encoding="utf-8") as f:
        json.dump(image_text_associations, f, ensure_ascii=False, indent=4)

    print(f"✅ Associations Image-Texte exportées : 📂 {images_json_path}")

    return text_csv_path, text_json_path, tables_json_path, images_json_path

# ==========================================
# ✅ CREATION D'UN FICHIER ZIP POUR TÉLÉCHARGEMENT
# ==========================================
def create_zip_file(zip_path, directory):
    """Crée une archive ZIP contenant tous les fichiers exportés."""
    shutil.make_archive(zip_path.replace(".zip", ""), 'zip', directory)
    print(f"✅ Fichier ZIP créé : 📂 {zip_path}")

# ==========================================
# 📌 📍 EXECUTION DU PROCESSUS COMPLET
# ==========================================
print("\n🔹 Exportation des données en CSV et JSON...")
exported_files = export_data(labeled_text_df, tables_data, image_text_associations)

print("\n🔹 Création d'un fichier ZIP pour simplifier le téléchargement...")
create_zip_file(zip_path, output_dir)

print(f"\n✅ **Téléchargement :** Allez dans **l'onglet 'Data' de Kaggle** et téléchargez le fichier `{zip_path}` 📂")



🔹 Exportation des données en CSV et JSON...
✅ Textes exportés : 📂 /kaggle/working/exported_data/labeled_text.csv
✅ Tables exportées : 📂 /kaggle/working/exported_data/tables.json
✅ Associations Image-Texte exportées : 📂 /kaggle/working/exported_data/image_text_associations.json

🔹 Création d'un fichier ZIP pour simplifier le téléchargement...
✅ Fichier ZIP créé : 📂 /kaggle/working/data_export.zip

✅ **Téléchargement :** Allez dans **l'onglet 'Data' de Kaggle** et téléchargez le fichier `/kaggle/working/data_export.zip` 📂
